### Appliying Python and pandas for data wrangling:

#### comp333, Assighnment 3, Nazli Ensafi, 40038607

The following workflos is the combination of what's suggested by Isaac Vidas in the video "Data mining and integration with Python" and what we learned in Lab8 tutorial Data Cleaning with OpenRefine for Ecology from datacarpentry.org

Workflow: 
1. Data Acquisition (Clean(leading or trailing spaces), Normalize(e.g. addresses, data types), Analyze(to see if it clean and normalyzes, if not go back to cleaning again)) 
2. Enrichment (aquiring more data from the existing data)
3. Entity Resolution(remove duplicate content) 
4. Combine(save the result in a new dataframe)

### 1. Data Aquisition:

In [49]:
# read the csv file
df = pd.read_csv('Portal_rodents_19772002_scinameUUIDs.csv', encoding='ISO-8859-1', low_memory=False)
print(df.head())


                              survey_id  recordID  mo  dy    yr  period  \
0  491ec41b-0a80-4ce9-b754-2983c6f38e71      6545   9  18  1982    62.0   
1  f280bade-4e9b-447d-afe0-9e6d45e6bfb0      5220   1  24  1982    54.0   
2  2b1b4a8a-c3e9-4781-a5f9-18b081383876     18932   8   7  1991   162.0   
3  e98e66c4-514b-45f7-9d95-336c05a72b21     20588   1  24  1993   179.0   
4  768cdd0d-93bc-4c74-9dcd-d1ed48b7501e      7020  11  21  1982    63.0   

                                plot_id  plot  note1  stake  ...  tag note2  \
0  4dc16022-f28d-4b9d-9062-c7bc3ad43362    13   13.0   36.0  ...  NaN   NaN   
1  dcbbd373-c22e-46ef-ae8b-ad88f5cf7475    20   13.0   27.0  ...  NaN   NaN   
2  1e87b11b-4795-4f41-bdff-2d5c4412be25    19   13.0   33.0  ...  NaN   NaN   
3  91829d58-c12e-47e8-ae31-185f46be21d2    12   13.0   41.0  ...  NaN   NaN   
4  f24f2db6-eadd-4de9-9d2f-95aea376c013    24   13.0   72.0  ...  NaN   NaN   

  ltag note3  prevrt  prevlet nestdir neststk note4 note5  
0  NaN   NaN  

#### 1.1. Clean

In [50]:
# Trim leading and trailing whitespace in all columns
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Print the modified DataFrame
print(df.head())

                              survey_id  recordID  mo  dy    yr  period  \
0  491ec41b-0a80-4ce9-b754-2983c6f38e71      6545   9  18  1982    62.0   
1  f280bade-4e9b-447d-afe0-9e6d45e6bfb0      5220   1  24  1982    54.0   
2  2b1b4a8a-c3e9-4781-a5f9-18b081383876     18932   8   7  1991   162.0   
3  e98e66c4-514b-45f7-9d95-336c05a72b21     20588   1  24  1993   179.0   
4  768cdd0d-93bc-4c74-9dcd-d1ed48b7501e      7020  11  21  1982    63.0   

                                plot_id  plot  note1  stake  ...  tag note2  \
0  4dc16022-f28d-4b9d-9062-c7bc3ad43362    13   13.0   36.0  ...  NaN   NaN   
1  dcbbd373-c22e-46ef-ae8b-ad88f5cf7475    20   13.0   27.0  ...  NaN   NaN   
2  1e87b11b-4795-4f41-bdff-2d5c4412be25    19   13.0   33.0  ...  NaN   NaN   
3  91829d58-c12e-47e8-ae31-185f46be21d2    12   13.0   41.0  ...  NaN   NaN   
4  f24f2db6-eadd-4de9-9d2f-95aea376c013    24   13.0   72.0  ...  NaN   NaN   

  ltag note3  prevrt  prevlet nestdir neststk note4 note5  
0  NaN   NaN  

#### 1.2. Normalize

In [51]:
print(df.dtypes)

survey_id            object
recordID              int64
mo                    int64
dy                    int64
yr                    int64
period              float64
plot_id              object
plot                  int64
note1               float64
stake               float64
species              object
scientificName       object
locality             object
JSON                 object
decimalLatitude     float64
decimalLongitude    float64
county               object
state                object
country              object
sex                  object
age                  object
reprod               object
testes               object
vagina               object
pregnant             object
nipples              object
lactation            object
hfl                 float64
wgt                 float64
tag                  object
note2                object
ltag                 object
note3                object
prevrt               object
prevlet             float64
nestdir             

In [52]:
# Get count of empty values in each column
empty_values = df.isna().sum()

# Print the empty values for each column
print(empty_values)

survey_id               0
recordID                0
mo                      0
dy                      0
yr                      0
period                  0
plot_id                 0
plot                    0
note1               31957
stake                  70
species              2015
scientificName      15318
locality            34874
JSON                35483
decimalLatitude     35489
decimalLongitude    35489
county              35453
state               35183
country             34871
sex                  2506
age                 20103
reprod              33898
testes              25857
vagina              33952
pregnant            34327
nipples             30521
lactation           35423
hfl                  4111
wgt                  3266
tag                  2324
note2               30965
ltag                 1901
note3               35533
prevrt               1780
prevlet              2071
nestdir             33718
neststk             30113
note4               34908
note5       

Replace empty values with proper data types:

In [53]:

import numpy as np

# Replace empty values in all columns with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Replace NaN values in numeric columns with 0
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(0)

# Replace NaN values in string columns with empty string
string_cols = df.select_dtypes(include=['object']).columns
df[string_cols] = df[string_cols].fillna('')

Change the datatypes of columns (period, note1, stake from "float 64" to "int 64"):

In [54]:

# Convert columns to int64 datatype
df['period'] = df['period'].astype('int64')
df['note1'] = df['note1'].astype('int64')
df['stake'] = df['stake'].astype('int64')

# Print the modified DataFrame
print(df.head())

                              survey_id  recordID  mo  dy    yr  period  \
0  491ec41b-0a80-4ce9-b754-2983c6f38e71      6545   9  18  1982      62   
1  f280bade-4e9b-447d-afe0-9e6d45e6bfb0      5220   1  24  1982      54   
2  2b1b4a8a-c3e9-4781-a5f9-18b081383876     18932   8   7  1991     162   
3  e98e66c4-514b-45f7-9d95-336c05a72b21     20588   1  24  1993     179   
4  768cdd0d-93bc-4c74-9dcd-d1ed48b7501e      7020  11  21  1982      63   

                                plot_id  plot  note1  stake  ... tag note2  \
0  4dc16022-f28d-4b9d-9062-c7bc3ad43362    13     13     36  ...             
1  dcbbd373-c22e-46ef-ae8b-ad88f5cf7475    20     13     27  ...             
2  1e87b11b-4795-4f41-bdff-2d5c4412be25    19     13     33  ...             
3  91829d58-c12e-47e8-ae31-185f46be21d2    12     13     41  ...             
4  f24f2db6-eadd-4de9-9d2f-95aea376c013    24     13     72  ...             

  ltag note3  prevrt  prevlet nestdir neststk note4 note5  
0                   

#### 1.3. Analyze

In [55]:
# Get count of empty values in each column
empty_values = df.isna().sum()

# Print the empty values for each column
print(empty_values)

survey_id           0
recordID            0
mo                  0
dy                  0
yr                  0
period              0
plot_id             0
plot                0
note1               0
stake               0
species             0
scientificName      0
locality            0
JSON                0
decimalLatitude     0
decimalLongitude    0
county              0
state               0
country             0
sex                 0
age                 0
reprod              0
testes              0
vagina              0
pregnant            0
nipples             0
lactation           0
hfl                 0
wgt                 0
tag                 0
note2               0
ltag                0
note3               0
prevrt              0
prevlet             0
nestdir             0
neststk             0
note4               0
note5               0
dtype: int64


Normalizng the scientificName column by clustering the values and making each cluster have a unique name across the cluster

### 3. Entity Resolution(remove duplicate content)

In [56]:
set(df['scientificName'])

{'',
 'Ammodramus savannarum',
 'Ammospermophilis harrisi',
 'Ammospermophilus harrisi',
 'Ammospermophilus harrisii',
 'Amphespiza bilineata',
 'Amphispiza bilineata',
 'Amphispiza bilineatus',
 'Amphispiza cilineata',
 'Amphispizo bilineata',
 'Baiomys taylori',
 'Calamospiza melanocorys',
 'Callipepla squamata',
 'Campylorhynchus brunneicapillus',
 'Chaetodipus baileyi',
 'Cnemidophorus tigris',
 'Cnemidophorus uniparens',
 'Crotalus scutalatus',
 'Crotalus viridis',
 'Dipodomys merriami',
 'Dipodomys ordii',
 'Dipodomys spectabilis',
 'Dipodomys\xa0sp.',
 'Emphispiza bilinata',
 'Onychomys leucogaster',
 'Onychomys torridus',
 'Onychomys\xa0sp.'}

In the current dataframe there are 27 unique values in the scientificName column, including an empty value. A quick investigation shows that the some values have typos and/or several versions stored. The correct set is the following, and we need to correct the data accordingly.

In [57]:
len(["Ammodramus savannarum", "Ammospermophilus harrisii", "Amphispiza bilineata", 
       "Baiomys taylori", "Calamospiza melanocorys", "Callipepla squamata", 
       "Campylorhynchus brunneicapillus", "Chaetodipus baileyi", "Cnemidophorus tigris", 
       "Aspidoscelis uniparens", "Crotalus scutulatus", "Crotalus viridis", 
       "Dipodomys merriami", "Dipodomys ordii", "Dipodomys spectabilis", "Dipodomys", 
       "Onychomys leucogaster", "Onychomys torridus", "Onychomys"])

19

Let's correct data reagarding the name "Amphispiza bilineata", "Ammospermophilus harrisii", "Onychomys" and "Dipodomys":

In [58]:
#import pandas as pd

# create a dictionary of old values and new value
replace_dict = {'Amphespiza bilineata': 'Amphispiza bilineata',
                'Amphispiza bilineatus': 'Amphispiza bilineata',
                'Amphispiza cilineata': 'Amphispiza bilineata',
                'Amphispizo bilineata': 'Amphispiza bilineata',
                'Emphispiza bilinata': 'Amphispiza bilineata',
               'Ammospermophilis harrisi': 'Ammospermophilus harrisii',
               'Ammospermophilus harrisi': 'Ammospermophilus harrisii',
               'Onychomys\xa0sp.': 'Onychomys',
               'Dipodomys\xa0sp.': 'Dipodomys'}

# replace the old values with the new value
df['scientificName'] = df['scientificName'].replace(replace_dict)

set(df['scientificName'])

{'',
 'Ammodramus savannarum',
 'Ammospermophilus harrisii',
 'Amphispiza bilineata',
 'Baiomys taylori',
 'Calamospiza melanocorys',
 'Callipepla squamata',
 'Campylorhynchus brunneicapillus',
 'Chaetodipus baileyi',
 'Cnemidophorus tigris',
 'Cnemidophorus uniparens',
 'Crotalus scutalatus',
 'Crotalus viridis',
 'Dipodomys',
 'Dipodomys merriami',
 'Dipodomys ordii',
 'Dipodomys spectabilis',
 'Onychomys',
 'Onychomys leucogaster',
 'Onychomys torridus'}

### 2. Enrichment (splitting scientificName into genus and species column)

In [59]:
# split the 'scientificName' column into two columns using str.split()
df[['genus', 'species']] = df['scientificName'].str.split(' ', n=1, expand=True)

# insert the new columns immediately after 'scientificName'
df.insert(loc=df.columns.get_loc('scientificName')+1, column='genus', value=df.pop('genus'))
df.insert(loc=df.columns.get_loc('scientificName')+2, column='species', value=df.pop('species'))

# print the updated DataFrame
print(df)


                                  survey_id  recordID  mo  dy    yr  period  \
0      491ec41b-0a80-4ce9-b754-2983c6f38e71      6545   9  18  1982      62   
1      f280bade-4e9b-447d-afe0-9e6d45e6bfb0      5220   1  24  1982      54   
2      2b1b4a8a-c3e9-4781-a5f9-18b081383876     18932   8   7  1991     162   
3      e98e66c4-514b-45f7-9d95-336c05a72b21     20588   1  24  1993     179   
4      768cdd0d-93bc-4c74-9dcd-d1ed48b7501e      7020  11  21  1982      63   
...                                     ...       ...  ..  ..   ...     ...   
35544  5e05640b-d193-482a-9e8d-10608f4e810b     35542  12  31  2002     295   
35545  a6bbdcbb-2c0a-4d3f-85a0-62a73270a108     35543  12  31  2002     295   
35546  67137ae7-0919-49c3-bd8f-4afda8162994     35544  12  31  2002     295   
35547  2bcb411b-9e81-42f6-a8db-aff0f863b564     35547  12  31  2002     295   
35548  d73a21e7-c8c2-4d00-83b6-015b597061e9     35549  12  31  2002     295   

                                    plot_id  plot  

In [60]:
df['genus']

0              Amphispiza
1              Amphispiza
2              Ammodramus
3              Ammodramus
4        Ammospermophilus
               ...       
35544                    
35545                    
35546                    
35547                    
35548                    
Name: genus, Length: 35549, dtype: object

In [61]:
df['species']

0         bilineata
1         bilineata
2        savannarum
3        savannarum
4          harrisii
            ...    
35544          None
35545          None
35546          None
35547          None
35548          None
Name: species, Length: 35549, dtype: object

### 4. Combine(save the result in a new dataframe)

In [62]:
# save the DataFrame to a CSV file
df.to_csv('clean_data.csv', index=False)